## 심층 신경망 구축 : 단계별

4 주차 과제 (part 1/2)에 오신 것을 환영합니다! 이전에 2층 신경망(단일 은닉층 포함)을 훈련했습니다. 이번 주에는 원하는 만큼의 레이어로 심층 신경망을 구축 할 것입니다!

* 이 노트북에서는 심층 신경망을 구축하는 데 필요한 모든 함수들을 구현합니다.
* 다음 과제에서는 이러한 함수를 사용하여 이미지 분류를 위한 심층 신경망을 구축합니다.

**이 과제 후 다음을 수행 할 수 있습니다**:

* ReLU와 같은 비선형 단위를 사용하여 모델 개선
* 더 깊은 신경망 구축 (1 개 이상의 숨겨진 레이어 포함)
* 사용하기 쉬운 신경망 클래스 구현


**표기법**:

* 위첨자 $ [l] $는 $ l^{th} $ 레이어와 관련된 수량을 나타냅니다.
  - 예 : $ a^{[L]} $는 $ L^{th} $ 레이어 활성화입니다. $ W^{[L]} $ 및 $ b^{[L]} $는 $ L^{th} $ 레이어 매개 변수입니다.
* 위첨자 $ (i) $는 $ i^{th} $ 학습자료와 관련된 수량을 나타냅니다.
  - 예 : $ x^{(i)} $는 $ i^{th} $ 학습 예입니다.
* 소문자 $ i $는 벡터의 $ i^{th} $ 항목을 나타냅니다.
  - 예 : $ a^{[l]}_i $는 $ l^{th} $ 레이어 활성화의 $ i^{th} $ 항목을 나타냅니다.

시작합시다!


### 1. 패키지

먼저 이 과제 중에 필요한 모든 패키지를 가져 오겠습니다.

* numpy는 Python을 사용한 과학 컴퓨팅을위한 기본 패키지입니다.
* matplotlib는 Python에서 그래프를 그리는 라이브러리입니다.
* dnn_utils는이 노트북에 필요한 몇 가지 기능을 제공합니다.
* testCases는 기능의 정확성을 평가하기 위한 몇 가지 테스트 케이스를 제공합니다.
* np.random.seed(1)는 모든 임의의 함수 호출을 일관되게 유지하는 데 사용됩니다. 귀하의 작업을 평가하는 데 도움이 됩니다. seed를 바꾸지 마십시오.

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4a import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload

np.random.seed(1)

### 2. 과제 개요

신경망을 구축하기 위해 몇 가지 "도우미 함수"을 구현합니다. 이 도우미 함수는 다음 과제에서 2-층 신경망과 L-층 신경망을 구축하는 데 사용됩니다. 구현할 각 작은 도우미 함수에는 필요한 단계를 안내하는 자세한 지침이 있습니다. 이 과제의 개요는 다음과 같습니다.

* 2-층 네트워크 및 $L$-층 신경망에 대한 매개 변수를 초기화 합니다.
* 순방향 전파 모듈을 구현합니다 (아래 그림에서 보라색으로 표시됨).
  - 레이어의 순방향 전파 단계의 LINEAR 부분을 완료합니다 (결과적으로 $Z^{[l]}$).
  - ACTIVATION 함수 (relu / sigmoid)을 제공합니다.
  - 이전 두 단계를 새로운 [LINEAR-> ACTIVATION] forward 함수로 결합합니다.
  - [LINEAR-> RELU] forward 함수를 L-1 번(레이어 1부터 L-1까지) 스택하고 끝에 [LINEAR-> SIGMOID]를 추가합니다 (최종 레이어 $ L $). 이것은 새로운 L_model_forward 함수를 제공합니다.
* 손실을 계산합니다.
* 역방향 전파 모듈을 구현합니다 (아래 그림에서 빨간색으로 표시됨).
  - 레이어의 역방향 전파 단계의 LINEAR 부분을 완료합니다.
  - ACTIVATION 함수 (relu_backward / sigmoid_backward)의 기울기를 제공합니다.
  - 이전 두 단계를 새로운 [LINEAR-> ACTIVATION] 역방향 함수로 결합합니다.
  - [LINEAR-> RELU]를 L-1 번 뒤로 스택하고 새 L_model_backward 함수에서 [LINEAR-> SIGMOID]를 뒤로 추가합니다.
* 마지막으로 매개 변수를 업데이트하십시오.

<img src="./images/final%20outline.png" width="600">

**주의** 모든 순방향 함수에는 해당하는 역방향 함수가 있습니다. 그래서 forward 모듈의 모든 단계에서 캐시에 일부 값을 저장하게됩니다. 캐시 된 값은 그라디언트 계산에 유용합니다. 역전파 모듈에서 캐시를 사용하여 그라디언트를 계산합니다. 이 과제는 이러한 각 단계를 수행하는 방법을 정확하게 보여줍니다.

### 3. 초기화
모델의 매개 변수를 초기화하는 두 개의 도우미 함수를 작성합니다. 첫 번째 함수는 2-층 모델의 매개 변수를 초기화하는 데 사용됩니다. 두 번째는 이 초기화 프로세스를 $ L $ 레이어로 일반화합니다.

#### 3.1. 2 층 신경망
**(1) 연습문제** : 2-층 신경망의 매개 변수를 만들고 초기화합니다.

**방법**:

* 모델의 구조는 LINEAR-> RELU-> LINEAR-> SIGMOID입니다.
* 가중치 행렬에 임의 초기화를 사용합니다. np.random.randn(shape) * 0.01을 올바른 모양으로 사용하십시오.
* 편향에 대해 0 초기화를 사용하십시오. np.zeros(shape)를 사용하십시오.

In [2]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 =  
    b1 =  
    W2 =  
    b2 =  
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td> [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]] </td> 
  </tr>

  <tr>
    <td> **b1**</td>
    <td>[[ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.01744812 -0.00761207]]</td>
  </tr>
  
  <tr>
    <td> **b2** </td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 3.2. L 층 신경망

더 깊은 L 계층 신경망의 초기화는 더 많은 가중치 행렬과 편향 벡터가 있기 때문에 더 복잡합니다. initialize_parameters_deep을 완료 할 때 각 레이어 간에 차원이 일치하는지 확인해야 합니다. $n^{[l]}$는 $l$ 레이어의 유닛 수입니다. 따라서 예를 들어 입력 $X$의 크기가 $ (12288, 209)$ ($ m = 209 $ 예제 포함)이면 :

<table style="width:100%" border="1px" >
<tr>
<td width="12%">  </td> 
<td width="18%"> Shape of W </td> 
<td width="14%"> Shape of b  </td> 
<td width="36%"> Activation </td>
<td width="20%"> Shape of Activation </td> 
<tr>
<tr>
<td> Layer 1 </td> 
<td> $(n^{[1]},12288)$ </td> 
<td> $(n^{[1]},1)$ </td> 
<td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
<td> $(n^{[1]},209)$ </td> 
<tr>
<tr>
<td> Layer 2 </td> 
<td> $(n^{[2]}, n^{[1]})$  </td> 
<td> $(n^{[2]},1)$ </td> 
<td> $Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
<td> $(n^{[2]}, 209)$ </td>
<tr>
<tr>
<td> $\vdots$ </td> 
<td> $\vdots$  </td> 
<td> $\vdots$  </td>
<td> $\vdots$</td> 
<td> $\vdots$  </td> 
<tr> 
<tr>
<td> Layer L-1 </td> 
<td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
<td> $(n^{[L-1]}, 1)$  </td> 
<td> $Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
<td> $(n^{[L-1]}, 209)$ </td> 
<tr>
<tr>
<td> Layer L </td> 
<td> $(n^{[L]}, n^{[L-1]})$ </td> 
<td> $(n^{[L]}, 1)$ </td>
<td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
<td> $(n^{[L]}, 209)$  </td> 
<tr>
</table>


파이썬에서 $ WX + b $를 계산할 때 브로드캐스팅을 수행한다는 것을 기억하십시오. 예를 들면 다음과 같습니다.

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$
그러면, $WX + b$ 는 다음과 같이 됩니다:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**(2) 연습문제** : L-층 신경망에 대한 초기화를 구현합니다.

**방법**:

* 모델의 구조는 [LINEAR-> RELU] $ \times $ (L-1)-> LINEAR-> SIGMOID입니다. 즉, ReLU 활성화 함수를 사용하는 $ L-1 $ 레이어와 시그모이드 활성화 함수가 있는 출력 레이어가 있습니다.
* 가중치 행렬에 임의 초기화를 사용합니다. np.random.randn(shape) * 0.01을 사용합니다.
* 편향에 대해 0 초기화를 사용합니다. np.zeros(shape)를 사용하십시오.
* 다른 레이어의 단위 수인 $ n^{[l]} $를 변수 layer_dims에 저장합니다. 예를 들어, 지난주 "평면 데이터 분류 모델"에 대한 layer_dims는 [2,4,1]이 됩니다. 입력이 2 개 있었는데, 하나에는 4 개의 은닉 유닛이 있고, 다른 하나는 1 개의 출력 유닛이 있는 출력 레이어였습니다. 이것은 W1의 모양이 (4,2), b1이 (4,1), W2가 (1,4)이고 b2가 (1,1)임을 의미합니다. 이제 이것을 $ L $ 레이어로 일반화합니다!
* 다음은 $ L = 1 $ (1 층 신경망)에 대한 구현입니다. 일반적인 경우 (L 층 신경망)를 구현하도록 영감을 줄 것입니다.

```
if L == 1:
      parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
      parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [3]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] =  
        parameters['b' + str(l)] =  
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td width="20%"> W1 </td>
    <td width="60%">[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>b1 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>W2 </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>b2 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4-순방향 전파 모듈
### 4.1-선형 순방향
이제 매개 변수를 초기화 했으므로 순방향 전파 모듈을 수행합니다. 나중에 모델을 구현할 때 사용할 몇 가지 기본 함수를 구현하는 것으로 시작합니다. 다음과 같은 순서로 세 가지 함수를 완료합니다.

* LINEAR
* LINEAR -> ACTIVATION 여기서 ACTIVATION은 ReLU 또는 Sigmoid입니다.
* [LINEAR-> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID (전체 모델)

선형 순방향 모듈(모든 예에 대하여 벡터화 된)은 다음 방정식을 계산합니다.

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$
여기에서  $A^{[0]} = X$.

**(3) 연습문제** : 순방향 전파의 선형 부분을 만듭니다.

**주의** : 이 유닛의 수학적 표현은 $ Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]} $입니다. np.dot( ) 이 유용 할 수도 있습니다. 차원이 일치하지 않으면 W.shape 을 출력하는 것이 도움이 될 수 있습니다.

In [4]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z =  
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [5]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**Expected output**:

<table style="width:40%">
  
  <tr>
    <td width="15%"> Z = </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>


### 4.2- 선형-활성화 순방향

이 노트북에서는 두 가지 활성화 함수를 사용합니다.

* **시그모이드** : $ \sigma(Z) = \sigma(WA + b) = \frac {1}{1 + e^{-(WA + b)}} $. 시그모이드 함수를 제공했습니다. 이 함수는 활성화 값 "a"와 "Z"를 포함하는 "cache"라는 두 항목을 반환합니다 (해당하는 역방향 함수에 제공 할 것입니다). 그것을 사용하려면 다음을 호출하십시오.

     A, activation_cache = sigmoid(Z)
     

* **ReLU** : ReLU의 수학 공식은 $ A = RELU(Z) = max(0, Z) $입니다. relu 함수를 제공했습니다. 이 함수는 활성화 값 "A"와 "Z"를 포함하는 "cache"라는 두 항목을 반환합니다 (해당하는 역방향 함수에 제공할 것입니다). 그것을 사용하려면 다음으로 호출하십시오.

     A, activation_cache = relu(Z)


편의를 위해 두 기능 (Linear 및 Activation)을 하나의 기능 (LINEAR-> ACTIVATION)으로 그룹화 할 것입니다. 따라서 LINEAR 앞으로 단계를 수행 한 다음 ACTIVATION 앞으로 단계를 수행하는 함수를 구현합니다.

**(3) 연습문제**  : LINEAR-> ACTIVATION 레이어의 순방향 전파를 구현합니다. 수학적 관계 : $ A ^ {[l]} = g (Z ^ {[l]}) = g (W ^ {[l]} A ^ {[l-1]} + b ^ {[l]} ) $ 여기서 활성화 "g"는 sigmoid() 또는 relu() 일 수 있습니다. linear_forward() 및 올바른 활성화 함수를 사용하십시오.

In [6]:

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache =  
        A, activation_cache =  
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache =  
        A, activation_cache = 
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [7]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Expected output**:
       
<table width="50%">
  <tr>
    <td width="25%">  With sigmoid: A  </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> With ReLU: A   </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>

참고 : 딥 러닝에서 "[LINEAR-> ACTIVATION]"계산은 신경망에서 두 계층이 아닌 단일 계층으로 계산됩니다.

### d) L 층 모델
$L$-층 Neural Net을 구현할 때 더 많은 편의를 위해 이전 함수 (RELU를 사용한 linear_activation_forward)를 $ L-1 $ 번 복제 한 다음 SIGMOID를 사용하여 linear_activation_forward를 한 번 수행하는 함수가 필요합니다.

<img src="./images/model_architecture_kiank.png">


**(4) 연습문제** : 위 모델의 순방향 전파를 구현합니다.

**지시** : 아래 코드에서 AL 변수는 $ A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]}) $. (때때로 Yhat이라고도합니다. 즉, $ \hat{Y} $입니다.)

**팁** :

* 이전에 작성한 함수 사용
* for 루프를 사용하여 [LINEAR-> RELU] (L-1) 번 복제
* "caches"목록에서 캐시를 추적하는 것을 잊지 마십시오. 목록에 새 값 c를 추가하려면 list.append(c)를 사용할 수 있습니다.


In [8]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache =  
        caches.append(    )
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache =   
    caches.append(    )
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [9]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


**Expected output**:

<table style="width:70%">
  <tr>
    <td width="30%"> AL </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td > Length of caches list  </td>
    <td > 3 </td> 
  </tr>
</table>

훌륭합니다! 이제 입력 X를 가져와 예측을 포함하는 행 벡터 $ A^{[L]} $를 출력하는 전체 순방향 전파가 있습니다. 또한 "caches"에 모든 중간 값을 기록합니다. $ A^{[L]} $를 사용하여 예측 비용을 계산할 수 있습니다.

### 5-비용 함수
이제 순방향 및 역방향 전파를 구현합니다. 모델이 실제로 학습 중인지 확인하기 위해 비용을 계산해야합니다.

**(5) 연습문제**: 다음 공식을 사용하여 교차 엔트로피 비용 $ J $를 계산합니다. 
$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

In [10]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost =  
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [11]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


**Expected Output**:

<table>
    <tr>
    <td>**cost** </td>
    <td> 0.2797765635793422</td> 
    </tr>
</table>

### 6-역방향 전파 모듈
순방향 전파와 마찬가지로 역 전파를위한 도우미 함수를 구현합니다. 역전파는 매개 변수에 대한 손실 함수의 기울기를 계산하는 데 사용됩니다.

조언:
<img src="./images/backprop_kiank.png">

이제 순방향 전파와 유사하게 세 단계로 역방향 전파를 빌드 할 것입니다.

* LINEAR 뒤로
* LINEAR-> ACTIVATION 뒤로 ACTIVATION이 ReLU 또는 시그모이드 활성화의 미분을 계산합니다.
* [LINEAR-> RELU] $ \ times $ (L-1)-> LINEAR-> SIGMOID 뒤로 (전체 모델)

#### 6.1-선형 역방향
$ l $ 레이어의 경우 선형 부분은 $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (활성화 후 ).

미분  $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$를 이미 계산했다고 가정합니다.$(dW^{[l]}, db^{[l]}, dA^{[l-1]})$를 얻고 싶습니다.

<img src="./images/linearback_kiank.png" width="200">

3 개의 출력  $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$는 입력 $dZ^{[l]}$를 사용하여 계산됩니다. 필요한 공식 :

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

**(6) 연습문제** : 위의 세 가지 공식을 사용하여 linear_backward()를 구현합니다.

In [12]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW =  
    db =  
    dA_prev =  
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [13]:

# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


**Expected Output**:
    
```
dA_prev = 
 [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = 
 [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = 
 [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
```


 
#### 6.2-역방향 선형 활성화
다음으로, linear_backward 및 linear_activation_backward 활성화를 위한 뒤로 단계라는 두 도우미 함수를 병합하는 함수를 만듭니다.

linear_activation_backward를 구현하는 데 도움을 주기 위해 두 개의 역방향 함수를 제공했습니다.

sigmoid_backward : SIGMOID 단위에 대한 역방향 전파를 구현합니다. 다음과 같이 호출 할 수 있습니다.
dZ = sigmoid_backward (dA, 활성화 _ 캐시)
relu_backward : RELU 유닛에 대한 역방향 전파를 구현합니다. 다음과 같이 호출 할 수 있습니다.
dZ = relu_backward (dA, 활성화 _ 캐시)
$ g (.) $가 활성화 함수이면 sigmoid_backward 및 relu_backward 계산

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$
 
 **(7): 연습문제** : LINEAR-> ACTIVATION 레이어에 대한 역전파를 구현합니다.

In [14]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ =  
        dA_prev, dW, db =  
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ =  
        dA_prev, dW, db =  
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [15]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Expected output with sigmoid:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 
  </tr>  
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr>   
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>

**Expected output with relu:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 
  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>


### 6.3-L 모델 영방향

이제 전체 네트워크에 대해 역방향 함수를 구현합니다. L_model_forward 함수를 구현할 때 각 반복에서 (X, W, b 및 z)를 포함하는 캐시를 저장했습니다. 역 전파 모듈에서는 이러한 변수를 사용하여 기울기를 계산합니다. 따라서 L_model_backward 함수에서 레이어 𝐿에서 시작하여 모든 숨겨진 레이어를 뒤로 반복합니다. 각 단계에서 레이어 𝑙에 대해 캐시 된 값을 사용하여 레이어 𝑙를 통해 역 전파합니다. 아래 그림 5는 역방향 패스를 보여줍니다.

<img src="./images/nm_backward.png">

**역전파 초기화** :이 네트워크를 통해 역 전파하려면 출력이  $A^{[L]} = \sigma(Z^{[L]})$ .이라는 것을 알고 있습니다. 따라서 코드는 dAL $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$ 를 계산해야 합니다. 이렇게 하려면 다음 공식을 사용하십시오 (심층 지식이 필요하지 않은 미적분을 사용하여 파생 됨).:
```
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```

그런 다음이 활성화 후 그라데이션 dAL을 사용하여 계속 뒤로 이동할 수 있습니다. 그림 5에서 볼 수 있듯이 이제 구현 한 LINEAR-> SIGMOID 역방향 함수에 dAL을 입력 할 수 있습니다 (L_model_forward 함수에 저장된 캐시 된 값을 사용함). 그 후 LINEAR-> RELU 역방향 함수를 사용하여 다른 모든 레이어를 반복하려면 for 루프를 사용해야 합니다. 각 dA, dW 및 db를 grads 사전에 저장해야합니다. 이렇게 하려면 다음 공식을 사용하십시오.:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

예를 들어 $ l = 3 $의 경우 $ dW^{[l]} $를 grads["dW3"]에 저장합니다.

**(9) 연습문제** : [LINEAR-> RELU] $ \ times $ (L-1)-> LINEAR-> SIGMOID 모델에 대한 역 전파를 구현합니다.

In [16]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = 
    ### END CODE HERE ###
        
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = 
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = 
    ### END CODE HERE ###
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = 
        dA_prev_temp, dW_temp, db_temp = 
        grads["dA" + str(l)] = 
        grads["dW" + str(l + 1)] = 
        grads["db" + str(l + 1)] = 
        ### END CODE HERE ###

    return grads

In [17]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


**Expected Output**

<table style="width:60%">
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td>
  </tr> 
</table>

#### 6.4-업데이트 매개 변수
이 섹션에서는 경사 하강 법을 사용하여 모델의 매개 변수를 업데이트합니다.
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

여기서 𝛼은 학습률입니다. 업데이트 된 매개 변수를 계산 한 후 매개 변수 사전에 저장합니다.

**(10) 연습문제** : 경사 하강 법을 사용하여 매개 변수를 업데이트하려면 update_parameters ()를 구현하십시오.

지침 :  $l = 1, 2, ..., L$.에 대해  $W^{[l]}$ 및 $b^{[l]}$ 마다 경사 하강 법을 사용하여 매개 변수를 업데이트합니다.

In [18]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] = 
        parameters["b" + str(l+1)] = 
    ### END CODE HERE ###
    return parameters

In [19]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**Expected Output**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>

### 7-결론
심층 신경망 구축에 필요한 모든 기능을 구현 한 것을 축하합니다!

긴 임무 였지만 앞으로 더 나아질 것입니다. 과제의 다음 부분이 더 쉽습니다.

다음 과제에서는이 모든 것을 결합하여 두 가지 모델을 구축합니다.

* 2-층 신경망
* L-층 신경망

실제로 이 모델을 사용하여 고양이 이미지와 고양이 이미지가 아닌 이미지를 분류합니다!